In [55]:
using LinearAlgebra, SparseArrays;

In [79]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [80]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [81]:
damping=0.85
function naive_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=zeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        if l>0 
            M[decodeNames[from], decodeNames[from]]=1-damping
            for to in list 
                M[decodeNames[to], decodeNames[from]]=damping/l
            end
        else
            M[decodeNames[from], decodeNames[from]]=1
        end
    end
    return (M, encodeNames, decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [82]:
damping=0.85
function sparse_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=spzeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        if l>0 
            M[decodeNames[from], decodeNames[from]]=1-damping
            for to in list 
                M[decodeNames[to], decodeNames[from]]=damping/l
            end
        else
            M[decodeNames[from], decodeNames[from]]=1
        end
    end
    return (M, encodeNames, decodeNames)
end

sparse_get_matrix (generic function with 1 method)

In [83]:
M, encode, decode=naive_get_matrix(input)

([0.15000000000000002 0.0 0.425 0.0; 0.0 0.15000000000000002 0.425 0.85; 0.425 0.85 0.15000000000000002 0.0; 0.425 0.0 0.0 0.15000000000000002], ["c", "b", "a", "d"], Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4))

In [84]:
M

4×4 Matrix{Float64}:
 0.15   0.0   0.425  0.0
 0.0    0.15  0.425  0.85
 0.425  0.85  0.15   0.0
 0.425  0.0   0.0    0.15

In [85]:
function naive(input)
    M, encode, decode = naive_get_matrix(input)
    Λ, X=eigen(M)
    n=size(Λ,1)
    ranks=X[:, n]
    ranks/=sum(ranks)
    sites=map(r-> (encode[r[1]],Float64(r[2])), enumerate(ranks))
    return sort(sites, by= x->x[2], rev=true)
end

naive (generic function with 1 method)

In [86]:
r=naive(input)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.39999999999999997)
 ("b", 0.30000000000000004)
 ("c", 0.19999999999999998)
 ("d", 0.10000000000000006)

In [87]:
function iterative(input, k)
    M, encode, decode = naive_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/n
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative (generic function with 1 method)

In [88]:
v=iterative(input, 40)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.39999999999999936)
 ("b", 0.29999999999999777)
 ("c", 0.20000000000000218)
 ("d", 0.10000000000000053)

In [89]:
function iterative_sparse(input, k)
    M, encode, decode = sparse_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/n
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative_sparse (generic function with 1 method)

In [90]:
v=iterative_sparse(input, 40)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.39999999999999936)
 ("b", 0.29999999999999777)
 ("c", 0.20000000000000218)
 ("d", 0.10000000000000053)

In [91]:
M, encode, decode = sparse_get_matrix(input)

(
 0.15    ⋅    0.425   ⋅ 
  ⋅     0.15  0.425  0.85
 0.425  0.85  0.15    ⋅ 
 0.425   ⋅     ⋅     0.15, ["c", "b", "a", "d"], Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4))